In [1]:
"""
Created on Wed Feb 26 17:01:56 2020

"""

""" IMPORTS """
import sys
import numpy as np
np.random.seed(1337)  # for reproducibility
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.constraints import *
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils

from binary_ops import binary_tanh as binary_tanh_op
from binary_layers import BinaryDense, BinaryConv2D

import h5py
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
from lambda_layers import *
from binary_ops import *
%matplotlib qt

In [2]:
""" FUNCTIION AND VARIABLE DEFINITIONS """
def binary_tanh(x):
    return binary_tanh_op(x)

H = 1.
kernel_lr_multiplier = 'Glorot'

# # nn
batch_size = 50
epochs = 20
channels = 1
img_rows = 30
img_cols = 30
filters = 32
kernel_size = (32, 32)
pool_size = (2, 2)
hidden_units = 128
classes = 10
use_bias = False

# # learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# # BN
epsilon = 1e-6
momentum = 0.9

# # dropout
p1 = 0.25
p2 = 0.5

hdf5_dir = Path("../data/hdf5/")

def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read
        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images= []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_vids.h5", "r+")

    images = np.array(file["/images"]).astype("float32")

    return images

def np_streak(x):
    input_dims = np.shape(x)
    output_shape = (input_dims[0],input_dims[1],input_dims[1]+input_dims[2],input_dims[3],input_dims[4])
    streak_tensor = np.zeros(output_shape)
    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            streak_tensor[i,j,j:(output_shape[3]+j),:,:] = x[i,j,:,:,:]
    #return streak_tensor
    return np.sum(streak_tensor,axis=1)

def mask(val,ims,mask):
    for i in range(np.shape(val)[0]):
        for j in range(np.shape(val)[1]):
            val[i,j,:,:] = ims[i,j,:,:] * mask
    return val



ims = read_many_hdf5(859)
# ims = np.ones((3943,30,32,32,1))
ims = np.reshape(ims, (-1,30,32,32,1))
ims = ims[:840]
# temp = np.zeros((1,32,32,1))
validate2 = np.zeros((840,30,32,32,1))
bk_temp = np.random.randint(0,2,(1,32,32,1))
validate2 = mask(validate2,ims,bk_temp)
ims2 = np_streak(validate2)


validate = ims

validate = validate / 255
ims2 = ims2 /255
ims = ims/255
#X_train, X_test, y_train, y_test = train_test_split(ims, validate, test_size=(1/3), random_state=42)
X_train, X_test, y_train, y_test = train_test_split(ims2, validate, test_size=(1/3), random_state=42)

MX_train, MX_test, My_train, My_test = train_test_split(ims,ims, test_size = 1/3, random_state = 42)

print(np.shape(X_test))
print(np.shape(X_train))

reduce_lr = ReduceLROnPlateau(monitor='val_loss',verbose=1, factor=0.5,
                              patience=50, min_lr=0.000001)
early_stopping = EarlyStopping(patience=90,verbose=1,restore_best_weights=True)   


def custom_loss(y_true, y_pred):

  ssim_loss = (1.0-tf.image.ssim(y_true,y_pred,1))/2.0
  mse_loss = K.mean(K.square(y_pred-y_true))
  #mse_loss = tf.keras.losses.mean_squared_error(y_true,y_pred)

  ssim_loss = 0.5*ssim_loss
  mse_loss = 0.5*mse_loss

  return ssim_loss + mse_loss

def ssim_loss(y_true,y_pred):  
    return (1.0-tf.image.ssim(y_true,y_pred,1))/2.0

def mse_loss(y_true,y_pred):
    return K.mean(K.square(y_pred-y_true))


(280, 62, 32, 1)
(560, 62, 32, 1)


In [3]:
""" VIDEO FUNCTIONS FOR CHECKING POST TRAINING"""

def get_mask(model,l=0):
    b = binarize(model.layers[l].weights[0])
    figb,axb = plt.subplots(1,1)
    axb.imshow(np.reshape(b,(32,32)),cmap="gray")
    
def show_video(y_pred,y_true,num):
    yp = np.reshape(y_pred,(-1,30,32,32))
    yt = np.reshape(y_true,(-1,30,32,32))
    fig,ax = plt.subplots(nrows=5,ncols=6,sharex=True,sharey=True)
    fig2,ax2 = plt.subplots(nrows=5,ncols=6,sharex=True,sharey=True)
    for row in range(5):
        for col in range(6):
            ax[row,col].imshow(yp[num][5*row+col],cmap="gray")
            ax2[row,col].imshow(yt[num][5*row+col],cmap="gray")

def show_all_videos(videos,rows,cols):
    yp = np.reshape(videos, (-1,30,32,32))
    fix3,ax3 = plt.subplots(nrows=rows, ncols = cols)
    for row in range(rows):
        for col in range(cols):
            ax3[row,col].imshow(yp[rows*row+col][3],cmap="gray")

In [4]:
""" FORWARD MODEL """

forward_model = Sequential()
forward_model.add(Input(shape=(30,32,32,1),batch_size = 40))
forward_model.add(TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                       data_format='channels_last',
                       H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                       padding='same', use_bias=use_bias, name='bin_conv_1')))
forward_model.add(Lambda(streak,output_shape=streak_output_shape))
forward_model.add(Lambda(integrate_ims, output_shape = integrate_ims_output_shape))
forward_model.add(Flatten())
forward_model.add(Dense(30720, activation = 'relu'))
forward_model.add(Reshape((30,32,32,1)))
forward_model.compile(optimizer = Nadam(0.0001), loss = custom_loss, metrics = ['mean_squared_error',mse_loss])
forward_model.summary()
forward_history = forward_model.fit(MX_train, My_train,
          batch_size = 40,epochs= 500,
          verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr, early_stopping])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda (Lambda)              (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_1 (Lambda)            (40, 62, 32, 1)           0         
_________________________________________________________________
flatten (Flatten)            (40, 1984)                0         
_________________________________________________________________
dense (Dense)                (40, 30720)               60979200  
_________________________________________________________________
reshape (Reshape)            (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
____________________________________________

KeyboardInterrupt: 

In [5]:
"""
Fine Tuning Parameters: 
Ratio between MSE and SSIM 
"""

ssim_alpha = 0.1
ssim_beta = 0.9
def variable_custom_loss(y_true, y_pred):
    global ssim_alpha, ssim_beta
    ssim_loss = (1.0-tf.image.ssim(y_true,y_pred,1))/2.0
    mse_loss = K.mean(K.square(y_pred-y_true))
    #mse_loss = tf.keras.losses.mean_squared_error(y_true,y_pred)

    ssim_loss = ssim_alpha*ssim_loss
    mse_loss = ssim_beta*mse_loss
    return ssim_loss + mse_loss


alpha = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
beta = [0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]

mse_losses = []
ssim_losses = []
mse_sd = []
ssim_sd = []

for i in range(9):
    ssim_alpha = alpha[i]
    ssim_beta = beta[i]
    inner_mse_losses=[]
    inner_ssim_losses=[]
    for j in range(1):
        MX_train, MX_test, My_train, My_test = train_test_split(ims,ims, test_size = 1/3)
        forward_model = Sequential()
        forward_model.add(Input(shape=(30,32,32,1),batch_size = 40))
        forward_model.add(TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                           data_format='channels_last',
                           H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                           padding='same', use_bias=use_bias, name='bin_conv_1')))
        forward_model.add(Lambda(streak,output_shape=streak_output_shape))
        forward_model.add(Lambda(integrate_ims, output_shape = integrate_ims_output_shape))
        forward_model.add(Flatten())
        forward_model.add(Dense(30720, activation = 'relu'))
        forward_model.add(Reshape((30,32,32,1)))
        forward_model.compile(optimizer = Nadam(0.0001), loss = variable_custom_loss, metrics = ['mean_squared_error',mse_loss,ssim_loss])
        forward_model.summary()
        forward_history = forward_model.fit(MX_train, My_train,
              batch_size = 40,epochs= 1,
              verbose=2,validation_data=(MX_test,My_test),callbacks=[reduce_lr])
        evals = forward_model.evaluate(MX_test,My_test)
        inner_mse_losses.append(evals[1])
        inner_ssim_losses.append(evals[2])
    mse_losses.append(np.mean(inner_mse_losses))
    ssim_losses.append(np.mean(inner_ssim_losses))
    mse_sd.append(np.std(inner_mse_losses))
    ssim_sd.append(np.std(inner_ssim_losses))


with open("training_logs.txt",'w') as file:
    for i in range(10):
        file.write("[ssim,mse]: " + "["+str(alpha[i])+","+str(beta[i])+"]"+ "MSE LOSS: " + str(mse_losses[i]) + " +/- " + str(mse_sd[i]) + + " SSIM LOSS: " + str(ssim_losses[i]) + " +/- " + str(ssim_sd[i]) + "\n")
    
    
    

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_2 (Lambda)            (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_3 (Lambda)            (40, 62, 32, 1)           0         
_________________________________________________________________
flatten_1 (Flatten)          (40, 1984)                0         
_________________________________________________________________
dense_1 (Dense)              (40, 30720)               60979200  
_________________________________________________________________
reshape_1 (Reshape)          (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_4 (Lambda)            (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_5 (Lambda)            (40, 62, 32, 1)           0         
_________________________________________________________________
flatten_2 (Flatten)          (40, 1984)                0         
_________________________________________________________________
dense_2 (Dense)              (40, 30720)               60979200  
_________________________________________________________________
reshape_2 (Reshape)          (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_6 (Lambda)            (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_7 (Lambda)            (40, 62, 32, 1)           0         
_________________________________________________________________
flatten_3 (Flatten)          (40, 1984)                0         
_________________________________________________________________
dense_3 (Dense)              (40, 30720)               60979200  
_________________________________________________________________
reshape_3 (Reshape)          (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_4 (TimeDist (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_8 (Lambda)            (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_9 (Lambda)            (40, 62, 32, 1)           0         
_________________________________________________________________
flatten_4 (Flatten)          (40, 1984)                0         
_________________________________________________________________
dense_4 (Dense)              (40, 30720)               60979200  
_________________________________________________________________
reshape_4 (Reshape)          (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_5 (TimeDist (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_10 (Lambda)           (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_11 (Lambda)           (40, 62, 32, 1)           0         
_________________________________________________________________
flatten_5 (Flatten)          (40, 1984)                0         
_________________________________________________________________
dense_5 (Dense)              (40, 30720)               60979200  
_________________________________________________________________
reshape_5 (Reshape)          (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_6 (TimeDist (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_12 (Lambda)           (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_13 (Lambda)           (40, 62, 32, 1)           0         
_________________________________________________________________
flatten_6 (Flatten)          (40, 1984)                0         
_________________________________________________________________
dense_6 (Dense)              (40, 30720)               60979200  
_________________________________________________________________
reshape_6 (Reshape)          (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_7 (TimeDist (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_14 (Lambda)           (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_15 (Lambda)           (40, 62, 32, 1)           0         
_________________________________________________________________
flatten_7 (Flatten)          (40, 1984)                0         
_________________________________________________________________
dense_7 (Dense)              (40, 30720)               60979200  
_________________________________________________________________
reshape_7 (Reshape)          (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (40, 30, 32, 32, 1)       1024      
_________________________________________________________________
lambda_16 (Lambda)           (40, 30, 62, 32, 1)       0         
_________________________________________________________________
lambda_17 (Lambda)           (40, 62, 32, 1)           0         
_________________________________________________________________
flatten_8 (Flatten)          (40, 1984)                0         
_________________________________________________________________
dense_8 (Dense)              (40, 30720)               60979200  
_________________________________________________________________
reshape_8 (Reshape)          (40, 30, 32, 32, 1)       0         
Total params: 60,980,224
Trainable params: 60,980,224
Non-trainable params: 0
__________________________________________

KeyboardInterrupt: 

In [15]:
# Check GPU memory, clear and recheck
from numba import cuda
gpu = cuda.select_device(0)
print('Selected GPU is:', gpu)
memory = cuda.current_context().get_memory_info()
print('Memory Status:', memory[0], 'free out of', memory[1], ',', (int(memory[0])/int(memory[1])*100), '% free')


Selected GPU is: <CUDA device 0 'b'Quadro M4000''>
Memory Status: 209754521 free out of 8589934592 , 2.4418640066869557 % free


In [14]:
K.clear_session()

In [ ]:
y_pred_ssim = model.predict(MX_train)
y_pred_ssim = tf.convert_to_tensor(y_pred_ssim)
y_true_ssim = tf.convert_to_tensor(My_train)

print("AVG SSIM: " + str(ssim_loss(y_pred_ssim, y_true_ssim).numpy().mean()))
print("AVG MSE: " + str(mse_loss(y_pred_ssim,y_true_ssim).numpy().mean()))
## sanity check

o = tf.convert_to_tensor(np.ones((30,30,1)))
z = tf.convert_to_tensor(np.zeros((30,30,1)))
ssim_loss(o,z)
tf.image.ssim(o,z,1)

In [ ]:
i = 0

for x in range(10):
    i += 1
    for z in range(10):
        print(i)